# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [38]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")


#https://stackoverflow.com/questions/59228094/module-not-found-error-no-module-named-config
#Stack overflow solution to use the config.py file outside of the "Weather_Database" folder created for this challenge
import sys
sys.path.append('../')

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [2]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df_data_to_load = "../Vacation_Search/WeatherPy_vacation.csv"
vacation_data_df = pd.read_csv(vacation_df_data_to_load)

# Display sample data
vacation_data_df.head()

,City_ID,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,0,Chiredzi,ZW,23.86,overcast clouds,-21.0500,31.6667,Nesbitt Arms Hotel
1,1,Rikitea,PF,26.52,scattered clouds,-23.1203,-134.9692,Chez Bianca & Benoit
2,7,Tomatlan,MX,29.66,clear sky,19.9333,-105.2500,Hotel Hacienda Vieja Tomatlan
3,10,Port Augusta,AU,25.15,clear sky,-32.5000,137.7667,Hotel Commonwealth
4,16,Ca Mau,VN,24.42,clear sky,9.1769,105.1500,Khách sạn Song Ngọc


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [3]:
# Configure the map plot
map_plot = vacation_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1.3,
    color = "City",
    hover_cols = ["City","Hotel Name","Country", "Weather Description"]
)
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Weather Description)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [4]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_data_df.loc[(vacation_data_df["City"] == "Cabo San Lucas")]
vacation_end = vacation_data_df.loc[(vacation_data_df["City"] == "Cabo San Lucas")]
vacation_stop1 = vacation_data_df.loc[(vacation_data_df["City"] == "Bucerias")]
vacation_stop2 = vacation_data_df.loc[(vacation_data_df["City"] == "Tomatlan")]
vacation_stop3 = vacation_data_df.loc[(vacation_data_df["City"] == "Acapulco")]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [5]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start,vacation_stop1,vacation_stop2,vacation_stop3,vacation_end])

# Display sample data
itinerary_df

,City_ID,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
26,95,Cabo San Lucas,MX,27.56,clear sky,22.8909,-109.9124,Comfort Rooms
44,178,Bucerias,MX,23.99,clear sky,20.7500,-105.3333,Royal Decameron Complex
2,7,Tomatlan,MX,29.66,clear sky,19.9333,-105.2500,Hotel Hacienda Vieja Tomatlan
52,208,Acapulco,MX,29.90,clear sky,16.8634,-99.8901,Hotel del Valle
26,95,Cabo San Lucas,MX,27.56,clear sky,22.8909,-109.9124,Comfort Rooms


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [37]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["Lat", "Lng"]].copy()


# Display sample data
waypoints_df

,Lat,Lng
26,22.8909,-109.9124
44,20.7500,-105.3333
2,19.9333,-105.2500
52,16.8634,-99.8901
26,22.8909,-109.9124


## Use GeoViews to create map that shows the four cities in the itinerary

In [39]:
# Configure the map plot by using the itineraty_df
waypoints_map = itinerary_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 3,
    color = "City",
    hover_cols = ["City","Hotel Name","Country", "Weather Description"]
)

In [40]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Weather Description)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [41]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [42]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '61b45929fb0f4bfc87e1701eefc418a6',
 'waypoints': '22.8909,-109.9124|20.75,-105.3333|19.9333,-105.25|16.8634,-99.8901|22.8909,-109.9124'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [43]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()

route_response

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [-109.9124, 22.8909], 'original_index': 0},
     {'location': [-105.3333, 20.75], 'original_index': 1},
     {'location': [-105.25, 19.9333], 'original_index': 2},
     {'location': [-99.8901, 16.8634], 'original_index': 3},
     {'location': [-109.9124, 22.8909], 'original_index': 4}],
    'units': 'metric',
    'distance': 8733096,
    'distance_units': 'meters',
    'time': 346401.047,
    'legs': [{'distance': 3462679,
      'time': 136117.781,
      'steps': [{'from_index': 0,
        'to_index': 4,
        'distance': 327,
        'time': 28.491,
        'instruction': {'text': 'Drive southeast on Avenida Leona Vicario.'}},
       {'from_index': 4,
        'to_index': 30,
        'distance': 1074,
        'time': 76.21,
        'instruction': {'text': 'Turn left onto Boulevard Lazaro Cardenas.'}},
       {'from_index': 30,
        'to_index': 57,
        'distance': 1155,
        '

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [44]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response["features"][0]["geometry"]["coordinates"]

legs

[[[-109.912519, 22.890833],
  [-109.912093, 22.890194],
  [-109.911563, 22.889428],
  [-109.911134, 22.888784],
  [-109.910801, 22.888351],
  [-109.910768, 22.888334],
  [-109.910726, 22.888322],
  [-109.910695, 22.888319],
  [-109.910669, 22.888323],
  [-109.910641, 22.888332],
  [-109.910619, 22.888345],
  [-109.910603, 22.888359],
  [-109.910581, 22.888382],
  [-109.910484, 22.888532],
  [-109.909568, 22.889925],
  [-109.909038, 22.890713],
  [-109.908951, 22.890836],
  [-109.908665, 22.891261],
  [-109.908359, 22.891723],
  [-109.908194, 22.891966],
  [-109.907663, 22.892819],
  [-109.907512, 22.893053],
  [-109.906922, 22.894006],
  [-109.906557, 22.894565],
  [-109.90616, 22.895151],
  [-109.906009, 22.895358],
  [-109.905813, 22.895606],
  [-109.905597, 22.895849],
  [-109.905396, 22.896044],
  [-109.905168, 22.89624],
  [-109.905079, 22.896294],
  [-109.905164, 22.896399],
  [-109.905225, 22.896483],
  [-109.905366, 22.896689],
  [-109.90566, 22.897079],
  [-109.906029, 22.8976

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [45]:
# Create and empty list to store the longitude of each step
longitude = []

# Create and empty list to store the latitude of step
latitude = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for leg in legs:
    for coordinate in leg:
        longitude.append(coordinate[0])
        latitude.append(coordinate[1])

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [46]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df["longitude"] = longitude
route_df["latitude"] = latitude

# Display sample data
route_df

,longitude,latitude
0,-109.912519,22.890833
1,-109.912093,22.890194
2,-109.911563,22.889428
3,-109.911134,22.888784
4,-109.910801,22.888351
...,...,...
77888,-109.913330,22.892021
77889,-109.913024,22.891583
77890,-109.912999,22.891545
77891,-109.912668,22.891056


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [50]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(route_df).opts(
    width = 800,
    height = 600,
    line_width = 2,
    color = "rebeccapurple"
)

In [51]:
# Display a composed plot by using the route_map and route_path objects
waypoints_map * route_path

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Weather Description)
   .Path.I   :Path   [longitude,latitude]